In [ ]:
res = !gcloud config get core/project
PROJECT_ID = res[0]
SERVICE_ACCOUNT = 'gemma-vertexai-chatbot@able-analyst-416817.iam.gserviceaccount.com'
from datetime import datetime
CONTAINER_IMAGE_NAME="gemma-chatbot"
GCP_REGION='us-central1'
TAG_NAME = 'latest'
KAGGLE_USERNAME='andrehpereh1'
KAGGLE_KEY='5859e39806d9456749dcbac685f04bc9'
CONTAINER_IMAGE_NAME_DATA_PREP = f"{CONTAINER_IMAGE_NAME}-data-preparation"
CONTAINER_IMAGE_NAME_FINE_TUNE = f"{CONTAINER_IMAGE_NAME}-fine-tunning"
CONTAINER_IMAGE_NAME_RUN_APP = f"{CONTAINER_IMAGE_NAME}-running-app"
CONTAINER_IMAGE_NAME_PIPELINE = f"{CONTAINER_IMAGE_NAME}-pipeline-app"
CONTAINER_IMAGE_RUNNING_APP = f"{CONTAINER_IMAGE_NAME}-running-app"
BUCKET_NAME = 'able-analyst-416817-chatbot-v1'
# BUCKET_URI = f"gs://{BUCKET_NAME}"
FINE_TUNE_FLAG=  False
EPOCHS=12
MODEL_NAME="gemma_2b_en"

substitutions="""
_CONTAINER_IMAGE_NAME_PIPELINE={},\
TAG_NAME={},\
_BUCKET_NAME={},\
_FINE_TUNE_FLAG={},\
_EPOCHS={},\
_MODEL_NAME={}
""".format(
           CONTAINER_IMAGE_NAME_PIPELINE,
           TAG_NAME,
           BUCKET_NAME,
           FINE_TUNE_FLAG,
           EPOCHS,
           MODEL_NAME
           ).strip()
print(substitutions)
!gcloud builds submit . --config "components/pipeline/cloudbuild.yaml" --substitutions {substitutions} --region={GCP_REGION}


In [ ]:
import os

CONTAINER_IMAGE_NAME="gemma-chatbot"
GCP_REGION = 'us-central1'
TAG_NAME = 'latest'

CONTAINER_IMAGE_NAME_PIPELINE = f"{CONTAINER_IMAGE_NAME}-pipeline-app"
CONTAINER_IMAGE_RUNNING_APP = f"{CONTAINER_IMAGE_NAME}-running-app"

#,_CONTAINER_IMAGE_RUNNING_APP={CONTAINER_IMAGE_RUNNING_APP}
substitutions=f"_CONTAINER_IMAGE_NAME_PIPELINE={CONTAINER_IMAGE_NAME_PIPELINE},TAG_NAME={TAG_NAME}"

In [ ]:
!gcloud builds submit . --config "./cloudbuil_cloudrun.yaml" --substitutions {substitutions} --region=us-central1

In [ ]:

!gcloud functions deploy hello_world_trigger \
--gen2 \
--region='us-central1' \
--runtime=python39 \
--source='./components/cloud_functions/' \
--entry-point='trigger_pipeline_cloud_function' \
--trigger-topic='your-pipeline-trigger-topic' \
--memory='2048MB'

In [ ]:
import kfp
kfp.__version__
dir(kfp.dsl)

In [ ]:
from components.cloud_functions.main import trigger_pipeline_cloud_function
import json

data = {
    "user_name": "andrehpereh",
    "files": [
        {"file_path": "gs://personalize-chatbots-v1/andrehpereh/input_data/WhatsApp Chat with Anki.txt", "filename": "WhatsApp Chat with Anki.txt"},
        {"file_path": "gs://personalize-chatbots-v1/andrehpereh/input_data/WhatsApp Chat with Ilse Flatmate.txt", "filename": "WhatsApp Chat with Ilse Flatmate.txt"},
        {"file_path": "gs://personalize-chatbots-v1/andrehpereh/input_data/WhatsApp Chat with Michael.txt", "filename": "WhatsApp Chat with Michael.txt"}
    ],
    "blob_folder": "andrehpereh/input_data",
    "model_name": "gemma_2b_en",
    "epochs": "12",
    "bucket_name": "personalize-chatbots-v1",
    "project_id": "able-analyst-416817"
}

# Convert dictionary to JSON string
json_data = json.dumps(data)

# Encode JSON string to bytes
data_bytes = json_data.encode('utf-8')

# Pass the encoded bytes to the function
trigger_pipeline_cloud_function()


In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "andrehpereh1"
os.environ['KAGGLE_KEY'] = "5859e39806d9456749dcbac685f04bc9"


In [ ]:
import importlib
import util
importlib.reload(util)
model_paths_and_config = util.get_model_paths_and_config(Config.MODEL_NAME)
model_paths_and_config

In [ ]:
Config.TRAIN_DATA_DIR, Config.BUCKET_NAME

In [ ]:
data = process_whatsapp_chat(Config.BUCKET_NAME, Config.TRAIN_DATA_DIR)

In [ ]:
model_paths_and_config

In [ ]:
finetuned_weights_path = finetune_gemma(data=data[:50], model_paths=model_paths_and_config, fine_tune_flag=False, model_name=Config.MODEL_NAME, rank_lora=Config.SEQUENCE_LENGTH, sequence_length=Config.SEQUENCE_LENGTH, epochs=Config.EPOCHS, batch_size=Config.BATCH_SIZE)

In [ ]:
from numba import cuda
device = cuda.get_current_device()
cuda.select_device(device.id)
cuda.close()

In [ ]:
type(finetuned_weights_path)
print(model_paths_and_config['finetuned_weights_path'])
print(model_paths_and_config['finetuned_model_dir'])

In [ ]:
!gsutil cp gs://able-analyst-416817-chatbot-v1/gemma_2b_en_raw/gemma_2b_en/model.weights.h5 ./test

In [ ]:
finetuned_weights_path.save_weights("gs://able-analyst-416817-chatbot-v1/gemma_2b_en_raw/gemma_2b_en/test/model.weights.h5")
finetuned_weights_path.preprocessor.tokenizer.save_assets("gs://able-analyst-416817-chatbot-v1/gemma_2b_en_raw/gemma_2b_en/test")

In [ ]:
output_dir = convert_checkpoints(
    weights_file=model_paths_and_config['finetuned_weights_path'],
    size=model_paths_and_config['model_size'],
    output_dir=model_paths_and_config['huggingface_model_dir'],
    vocab_path=model_paths_and_config['finetuned_vocab_path'],
)


In [ ]:
output_dir=model_paths_and_config['huggingface_model_dir']

In [ ]:
model_paths_and_config, Config.BUCKET_NAME
bucket_name, blob_name = os.path.dirname("gs://able-analyst-416817-chatbot-v1/gemma_2b_en_raw/gemma_2b_en/model.weights.h5").lstrip("gs://").split("/", 1) 
bucket_name, blob_name

In [ ]:
os.path.basename("gs://able-analyst-416817-chatbot-v1/gemma_2b_en_raw/gemma_2b_en/model.weights.h5")

In [ ]:
destination_path = upload2bs(local_directory = output_dir, bucket_name = Config.BUCKET_NAME, destination_subfolder = model_paths_and_config['deployed_model_blob'])

In [ ]:
import importlib
import util
importlib.reload(util)

util.download_all_from_blob(Config.BUCKET_NAME, "gemma_2b_en_raw/gemma_2b_en", './')

In [ ]:
Config.BUCKET_NAME
model_paths_and_config['huggingface_model_dir']

In [ ]:
model_paths_and_config['deployed_model_blob']

In [ ]:
aiplatform.init(project=Config.PROJECT_ID, location=Config.REGION, staging_bucket=Config.BUCKET_URI)

In [ ]:
import datetime
VLLM_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve:20240220_0936_RC01"


def get_job_name_with_datetime(prefix: str) -> str:
    suffix = datetime.datetime.now().strftime("_%Y%m%d_%H%M%S")
    return f"{prefix}{suffix}"


def deploy_model_vllm(
    model_name: str,
    model_uri: str,
    service_account: str,
    machine_type: str = "g2-standard-8",
    accelerator_type: str = "NVIDIA_L4",
    accelerator_count: int = 1,
    max_model_len: int = 8192,
    dtype: str = "bfloat16",
) -> tuple[aiplatform.Model, aiplatform.Endpoint]:
    # Upload the model to "Model Registry"
    job_name = get_job_name_with_datetime(model_name)
    vllm_args = [
        "--host=0.0.0.0",
        "--port=7080",
        f"--tensor-parallel-size={accelerator_count}",
        "--swap-space=16",
        "--gpu-memory-utilization=0.95",
        f"--max-model-len={max_model_len}",
        f"--dtype={dtype}",
        "--disable-log-stats",
    ]
    model = aiplatform.Model.upload(
        display_name=job_name,
        artifact_uri=model_uri,
        serving_container_image_uri=VLLM_DOCKER_URI,
        serving_container_command=["python", "-m", "vllm.entrypoints.api_server"],
        serving_container_args=vllm_args,
        serving_container_ports=[7080],
        serving_container_predict_route="/generate",
        serving_container_health_route="/ping",
    )

    # Deploy the model to an endpoint to serve "Online predictions"
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")
    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=service_account,
    )

    return model, endpoint

In [ ]:
# Set up artificially since the model was already in a bucket
model_paths_and_config['deployed_model_uri'] = 'gs://able-analyst-416817-chatbot-v1/gemma_2b_en/20240314162107'
model_paths_and_config['model_name_vllm']

In [ ]:
max_model_len = 2048

model, endpoint = deploy_model_vllm(
    model_name=model_paths_and_config['model_name_vllm'],
    model_uri=model_paths_and_config['deployed_model_uri'],
    service_account=Config.SERVICE_ACCOUNT,
    machine_type=model_paths_and_config['machine_type'],
    accelerator_type=model_paths_and_config['accelerator_type'],
    accelerator_count=model_paths_and_config['accelerator_count'],
    max_model_len=max_model_len,
)

In [ ]:
TEST_EXAMPLES = [
    "What is the plan for tonight?",
    "What would you like to drink?",
    "Are you coming tonight?"
]

# Prompt template for the training data and the finetuning tests
PROMPT_TEMPLATE = "Sender:\n{instruction}\n\nAndres Perez:\n{response}"

TEST_PROMPTS = [
    PROMPT_TEMPLATE.format(instruction=example, response="")
    for example in TEST_EXAMPLES
]

def test_vertexai_endpoint(endpoint: aiplatform.Endpoint):
    for question, prompt in zip(TEST_EXAMPLES, TEST_PROMPTS):
        instance = {
            "prompt": prompt,
            "max_tokens": 56,
            "temperature": 0.0,
            "top_p": 1.0,
            "top_k": 1,
            "raw_response": True,
        }
        response = endpoint.predict(instances=[instance])
        output = response.predictions[0]
        print(f"{question}\n{output}\n{'- '*40}")


test_vertexai_endpoint(endpoint)

In [ ]:
res = !gcloud config get core/project
PROJECT_ID = res[0]
SERVICE_ACCOUNT = 'gemma-vertexai-chatbot@able-analyst-416817.iam.gserviceaccount.com'
from datetime import datetime
CONTAINER_IMAGE_NAME="gemma-chatbot"
GCP_REGION='us-central1'
IMAGE_NAME="gemma-chatbot"
TAG_NAME = 'latest'
KAGGLE_USERNAME='andrehpereh1'
KAGGLE_KEY='5859e39806d9456749dcbac685f04bc9'

In [ ]:
SUBSTITUTIONS="""
_CONTAINER_IMAGE_NAME={},\
TAG_NAME={}\
""".format(
           f"{CONTAINER_IMAGE_NAME}-data-preparation",
           TAG_NAME, 
           ).strip()
print(SUBSTITUTIONS)


In [ ]:
# Runs the data_preparation component image. (Development, when tested should be moved to the main cloudbuild in the project folder)
# Pay attention to the "." after summit. Might need some changes when move to the master pipeline.
!gcloud builds submit . --timeout=15m --config "components/data_preparation/cloudbuild.yaml" --substitutions {SUBSTITUTIONS} --region={GCP_REGION}
# DO not forget the tag
#!docker run gcr.io/able-analyst-416817/gemma-chatbot-data-preparation:latest data_ingestion.py --bucket-name 'able-analyst-416817-chatbot-v1' --directory 'input_data/andrehpereh'

In [ ]:
SUBSTITUTIONS="""
_CONTAINER_IMAGE_NAME={},\
_KAGGLE_USERNAME={},\
_KAGGLE_KEY={},\
TAG_NAME={}\
""".format(
           f"{CONTAINER_IMAGE_NAME}-fine-tunning",
           KAGGLE_USERNAME,
           KAGGLE_KEY,
           TAG_NAME, 
           ).strip()
print(SUBSTITUTIONS)

# Builds image
!gcloud builds submit . --config "components/fine_tunning/cloudbuild.yaml" --substitutions {SUBSTITUTIONS} --region={GCP_REGION}


In [ ]:
import json

data = ["Sender: FoooodddAndres Perez: Coming :)", "Sender: Can I maybe borrow your iron? Andres Perez: It\'s not my iron But yeah haha Or is it?"]
model_paths = """{"finetuned_model_dir": "./gemma_2b_en", "finetuned_weights_path": "./gemma_2b_en/model.weights.h5"}"""
print(len(data))
model_paths = json.dumps(model_paths)
#!docker run gcr.io/able-analyst-416817/gemma-chatbot-fine-tunning:latest trainer.py --data {data} --model-paths {model_paths}

In [ ]:
!python ./components/fine_tunning/trainer.py --data {data} --model-paths {model_paths}

In [ ]:
model_paths_and_config['huggingface_model_dir']

In [ ]:
!python ./components/fine_tunning/conversion_function.py --weights-file {model_paths_and_config['finetuned_weights_path']} --size {model_paths_and_config['model_size']} --vocab-path {model_paths_and_config['finetuned_vocab_path']} --output-dir {model_paths_and_config['huggingface_model_dir']}

In [ ]:
output_dir = convert_checkpoints(
    weights_file=model_paths_and_config['finetuned_weights_path'],
    size=model_paths_and_config['model_size'],
    output_dir=model_paths_and_config['huggingface_model_dir'],
    vocab_path=model_paths_and_config['finetuned_vocab_path'],
)

In [ ]:
import json
SUBSTITUTIONS="""
_CONTAINER_IMAGE_NAME={},\
TAG_NAME={}\
""".format(
           f"{CONTAINER_IMAGE_NAME}-experimental",
           TAG_NAME,
           ).strip()
print(SUBSTITUTIONS)
data = '"[\\"Sender: FoooodddAndres Perez: Coming :)\\", \\"Sender: Can I maybe borrow your iron? Andres Perez: It\'s not my iron But yeah haha Or is it?\\"]"'
model_paths = """{"finetuned_model_dir": "./gemma_2b_en", "finetuned_weights_path": "./gemma_2b_en/model.weights.h5"}"""
data_json = json.dumps(data)
model_paths_json = json.dumps(model_paths)
#!gcloud builds submit . --timeout=15m --config "components/experimental/cloudbuild.yaml" --substitutions {SUBSTITUTIONS} --region={GCP_REGION}
!docker run gcr.io/able-analyst-416817/gemma-chatbot-experimental:latest experimental.py {data_json} {model_paths_json}


In [ ]:
#Re-runs the image to restart the website, service account might be needed with this one.  (Development, when tested should be moved to the main cloudbuild in the project folder)
SUBSTITUTIONS="""
_CONTAINER_IMAGE_NAME={},\
_GCP_REGION={},\
TAG_NAME={}\
""".format(
           f"{CONTAINER_IMAGE_NAME}-running-app",
           GCP_REGION,
           TAG_NAME, 
           ).strip()
print(SUBSTITUTIONS)
!gcloud builds submit . --timeout=15m --config "components/app_flask/cloudbuild.yaml" --substitutions {SUBSTITUTIONS} --region={GCP_REGION}
#!gcloud builds submit . --timeout=15m --config "cloudbuild.yaml" --substitutions {SUBSTITUTIONS} --region={GCP_REGION}


In [ ]:
SUBSTITUTIONS="""
_CONTAINER_IMAGE_NAME={},\
_GCP_REGION={},\
TAG_NAME={}\
""".format(
           f"{CONTAINER_IMAGE_NAME}-running-app",
           GCP_REGION,
           TAG_NAME, 
           ).strip()
print(SUBSTITUTIONS)
!gcloud builds submit . --timeout=15m --config "components/app_flask/cloudbuild.yaml" --substitutions {SUBSTITUTIONS} --region={GCP_REGION}
#!gcloud builds submit . --timeout=15m --config "cloudbuild.yaml" --substitutions {SUBSTITUTIONS} --region={GCP_REGION}

In [ ]:
from config import Config
from util import get_model_paths_and_config, upload2bs

res = !gcloud config get core/project
PROJECT_ID = res[0]
from google.cloud import compute_v1

project = PROJECT_ID  # Replace with your project ID

# Regions to consider
regions = ["us-central1", "europe-west4", "asia-east1"]  

client = compute_v1.AcceleratorTypesClient()

for region in regions:
    zone_client = compute_v1.ZonesClient()
    all_zones = zone_client.list(project=project)

    zone_list = [zone for zone in all_zones if zone.region == f"regions/{region}"]

    for zone in zone_list:
        result = client.describe(
            project=project, zone=zone.name, accelerator_type="nvidia-l4"
        )
        # If the result isn't an error, the 'nvidia-l4' type is available
        GCP_REGION = region  # Update your region variable
        break 

In [ ]:
from config import Config
from util import get_model_paths_and_config, upload2bs

res = !gcloud config get core/project
PROJECT_ID = res[0]
SERVICE_ACCOUNT = 'gemma-vertexai-chatbot@able-analyst-416817.iam.gserviceaccount.com'
from datetime import datetime
CONTAINER_IMAGE_NAME="gemma-chatbot"
GCP_REGION='us-central1'
IMAGE_NAME="gemma-chatbot"
TAG_NAME = 'latest'
KAGGLE_USERNAME='andrehpereh1'
KAGGLE_KEY='5859e39806d9456749dcbac685f04bc9'
model_paths_and_config = get_model_paths_and_config(Config.MODEL_NAME)
print(model_paths_and_config)
from google.cloud import aiplatform as vertexai
from kfp import dsl
from kfp.dsl import OutputPath, Artifact, InputPath
from kfp import compiler
# from google_cloud_pipeline_components.v1.custom_job import create_custom_training_job_from_component
# from google_cloud_pipeline_components.v1.custom_job import CustomTrainingJobOp


@dsl.component(
  base_image ='gcr.io/able-analyst-416817/gemma-chatbot-data-preparation:latest'
)
def process_whatsapp_chat_op(
  bucket_name: str,
  directory: str,
  dataset_path: OutputPath('Dataset')
):
    import data_ingestion
    import json
    formatted_messages = data_ingestion.process_whatsapp_chat(bucket_name, directory)
    with open(dataset_path, 'w') as f:
        json.dump(formatted_messages, f)


@dsl.component(
  base_image = 'gcr.io/able-analyst-416817/gemma-chatbot-fine-tunning:latest'
)
def fine_tunning(
  dataset_path: InputPath('Dataset'),
  model_paths: dict,
  finetuned_weights_path: OutputPath('Model'),
):
    # import test_container
    import trainer
    import json
    with open(dataset_path, 'r') as f:
        dataset = json.load(f)
    # finetuned_weights_path = test_container.add_test(dataset, model_paths)
    finetuned_weights_path = trainer.finetune_gemma(dataset, model_paths, False)
    print("Si funciono este pedo", finetuned_weights_path)
    # return finetuned_weights_path

@dsl.component(
  base_image = 'gcr.io/able-analyst-416817/gemma-chatbot-fine-tunning:latest'
)
def convert_checkpoints_op(
  finetuned_weights_path: InputPath('Model'),
  size: str,
  output_dir: str,
  vocab_path: str,
  converted_fined_tuned_path: OutputPath('Model')
):
    import conversion_function
    converted_fined_tuned_path = conversion_function.convert_checkpoints(
        weights_file=finetuned_weights_path,
        size=size,
        output_dir=output_dir,
        vocab_path=vocab_path,
    )



# TRAINER_ARGS = ["trainer.finetune_gemma.py"] + [ "--data-dir", dataset_path, "--hptune-dict", model_paths]
# custom_job_task = CustomTrainingJobOp(
#     display_name="model-training-CustomTrainingJobOp",
#     worker_pool_specs=[
#         {
#             "containerSpec": {
#                 "args": "trainer.finetune_gemma.py",
#                 "imageUri": "gcr.io/able-analyst-416817/gemma-chatbot-fine-tunning:latest",
#             },
#             "replicaCount": "1",
#             "machineSpec": {
#                 "machineType": "n1-standard-16",
#                 "accelerator_type": aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
#                 "accelerator_count": 2,
#             },
#         }
#     ],
# )


# Convert the above component into a custom training job, if set_memory_limit and add are added in the pipeline, this is not needed.
# custom_training_job_fine_tune = create_custom_training_job_from_component(
#    fine_tunning,
#    display_name = 'DISPLAY_NAME',
#    machine_type = 'g2-standard-12',
#    accelerator_type='NVIDIA_L4',
#    accelerator_count=1,
#    replica_count=1
#)




@dsl.pipeline
def pythagorean(
    bucket_name: str = "able-analyst-416817-chatbot-v1", directory: str = "input_data/andrehpereh", 
    model_paths: dict=model_paths_and_config
):
    whatup = process_whatsapp_chat_op(bucket_name = bucket_name, directory = directory)
    # trainer = custom_training_job_fine_tune(dataset_path=whatup.outputs['dataset_path'], model_paths=model_paths)
    # print(trainer.outputs['gcp_resources'])
    trainer = fine_tunning(dataset_path=whatup.outputs['dataset_path'], model_paths=model_paths)
    trainer.set_memory_limit("28G").set_cpu_limit('4.0m') # .set_accelerator_limit(1).add_node_selector_constraint("NVIDIA_TESLA_T4")
    converted = convert_checkpoints_op(
        finetuned_weights_path=trainer.outputs['finetuned_weights_path'], size=model_paths_and_config['model_size'],
        output_dir=model_paths_and_config['huggingface_model_dir'],
        vocab_path=model_paths_and_config['finetuned_vocab_path']
    )
converted.outputs['converted_fined_tuned_path']

compiler.Compiler().compile(
    pipeline_func=pythagorean, package_path="test-whatsapp.json"
)
vertexai.init(project=PROJECT_ID, location="us-central1")
vertex_pipelines_job = vertexai.pipeline_jobs.PipelineJob(
    display_name="test-whatsapp",
    template_path="test-whatsapp.json"
)
#vertex_pipelines_job.worker_pool_specs = worker_pool_specs 
vertex_pipelines_job.run()

In [ ]:
!bq mk chatbot

In [ ]:
from google.cloud import bigquery
import os

# Construct a BigQuery client object
os.environ['PROJECT_ID'] = 'able-analyst-416817'
client = bigquery.Client(os.environ.get('PROJECT_ID'))

# Define your dataset and table information
project_id = os.environ.get('PROJECT_ID')

In [ ]:

dataset_id = "chatbot"
table_id = "users" 

# Schema definition
schema = [
    # bigquery.SchemaField("user_id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("email", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("password_hash", "STRING", mode="REQUIRED"),
    #bigquery.SchemaField("salt", "STRING", mode="REQUIRED"),
    #bigquery.SchemaField("created_at", "TIMESTAMP", mode="REQUIRED"),
    #bigquery.SchemaField("last_login", "TIMESTAMP", mode="NULLABLE") 
]

# Create a table object
table = bigquery.Table(project_id + "." + dataset_id + "." + table_id, schema=schema)

# Create the table in BigQuery
table = client.create_table(table)  
print("Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id))


In [ ]:
dataset_id = "chatbot"
table_id = "user_training_status" 

# Schema definition
schema = [
    bigquery.SchemaField("email", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("training_status", "BOOL", mode="REQUIRED")
]

# Create a table object
table = bigquery.Table(project_id + "." + dataset_id + "." + table_id, schema=schema)

# Create the table in BigQuery
table = client.create_table(table)  
print("Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id))